In [0]:
df = (
    spark.read
    .format("csv")  # or "parquet", "json", etc.
    .option("header", "true")
    .load("/Volumes/abc/abc_dw_bronze/dataset/raw/Dataset for Supply chain Dataset.csv")
)
df.printSchema()

root
 |-- purchaseorder: string (nullable = true)
 |-- po_itemnumber: string (nullable = true)
 |-- purchaserequisition: string (nullable = true)
 |-- pr_itemnumber: string (nullable = true)
 |-- po_serialnumber: string (nullable = true)
 |-- po_returnsitem: string (nullable = true)
 |-- po_purchasingdoctype: string (nullable = true)
 |-- po_purchasingdoctypedesc: string (nullable = true)
 |-- po_purchasingdocdate: string (nullable = true)
 |-- po_deletionindicator: string (nullable = true)
 |-- po_purchasingorg: string (nullable = true)
 |-- po_purchasingorgdesc: string (nullable = true)
 |-- po_companycode: string (nullable = true)
 |-- po_companycodedesc: string (nullable = true)
 |-- po_purchasinggroup: string (nullable = true)
 |-- po_purchasinggroupdesc: string (nullable = true)
 |-- po_vendornumber: string (nullable = true)
 |-- po_vendorname: string (nullable = true)
 |-- po_vendoraccountgroup: string (nullable = true)
 |-- po_vendoraccountgroupdesc: string (nullable = true)
 |

In [0]:
df = (
    spark.read
    .format("csv")  # or "parquet", "json", etc.
    .option("header", "true")
    .load("/Volumes/abc/abc_dw_bronze/dataset/raw/Dataset for Supply chain Dataset.csv")
)
df.printSchema()

In [0]:
files = dbutils.fs.ls("/Volumes/abc/abc_dw_bronze/dataset/raw/")
display(files)

path,name,size,modificationTime
dbfs:/Volumes/abc/abc_dw_bronze/dataset/raw/Dataset for Supply chain Dataset.csv,Dataset for Supply chain Dataset.csv,456965,1763832541000


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import *

# ---------------------------------------------------------------------
# 1. DEFINE SCHEMA
# ---------------------------------------------------------------------

po_schema = StructType([
     StructField("purchaseorder", StringType(), True),
    StructField("po_itemnumber", StringType(), True),
    StructField("purchaserequisition", StringType(), True),
    StructField("pr_itemnumber", StringType(), True),
    StructField("po_serialnumber", StringType(), True),
    StructField("po_returnsitem", StringType(), True),
    StructField("po_purchasingdoctype", StringType(), True),
    StructField("po_purchasingdoctypedesc", StringType(), True),
    StructField("po_purchasingdocdate", StringType(), True),
    StructField("po_deletionindicator", StringType(), True),
    StructField("po_purchasingorg", StringType(), True),
    StructField("po_purchasingorgdesc", StringType(), True),
    StructField("po_companycode", StringType(), True),
    StructField("po_companycodedesc", StringType(), True),
    StructField("po_purchasinggroup", StringType(), True),
    StructField("po_purchasinggroupdesc", StringType(), True),
    StructField("po_vendornumber", StringType(), True),
    StructField("po_vendorname", StringType(), True),
    StructField("po_vendoraccountgroup", StringType(), True),
    StructField("po_vendoraccountgroupdesc", StringType(), True),
    StructField("po_supplierclassification", StringType(), True),
    StructField("po_countrykey", StringType(), True),
    StructField("po_localorinternational", StringType(), True),
    StructField("po_plant", StringType(), True),
    StructField("po_plantdesc", StringType(), True),
    StructField("po_storageloc", StringType(), True),
    StructField("po_createdon", StringType(), True),
    StructField("po_createdby_name", StringType(), True),
    StructField("po_documentyear", StringType(), True),
    StructField("po_accountassignmentcat", StringType(), True),
    StructField("po_accountassignmentcatdesc", StringType(), True),
    StructField("po_accntassignmentcatvaluedesc", StringType(), True),
    StructField("po_version", StringType(), True),
    StructField("po_incoterms1", StringType(), True),
    StructField("po_processingstatuscode", StringType(), True),
    StructField("po_processingstatus", StringType(), True),
    StructField("po_postingdate", StringType(), True),
    StructField("po_approvaldate", StringType(), True),
    StructField("po_paymentterms", StringType(), True),
    StructField("po_paymenttermsdesc", StringType(), True),
    StructField("material", StringType(), True),
    StructField("materialdesc", StringType(), True),
    StructField("materialgroup", StringType(), True),
    StructField("materialgroupdesc", StringType(), True),
    StructField("materialtype", StringType(), True),
    StructField("materialtypedesc", StringType(), True),
    StructField("po_deliverydate", StringType(), True),
    StructField("po_unitofmeasure", StringType(), True),
    StructField("po_orderpriceunit", StringType(), True),
    StructField("po_orderquantity", StringType(), True),
    StructField("po_priceunit", StringType(), True),
    StructField("po_netprice", StringType(), True),
    StructField("po_currency", StringType(), True),
    StructField("po_netordervalue", StringType(), True),
    StructField("po_netamount", StringType(), True),
    StructField("po_deliveredqty", StringType(), True),
    StructField("po_deliveredqtyvalue", StringType(), True),
    StructField("po_pendingdeliveryqty", StringType(), True),
    StructField("po_pendingdeliveryvalue", StringType(), True),
    StructField("po_deliverycompleteindicator", StringType(), True),
    StructField("po_paymentstatus", StringType(), True),
    StructField("po_advancepayment", StringType(), True),
    StructField("po_paymentdate", StringType(), True),
    StructField("po_headerstatus", StringType(), True),
    StructField("po_itemstatus", StringType(), True),
    StructField("pr_documenttype", StringType(), True),
    StructField("pr_creationdate", StringType(), True),
    StructField("pr_createdby_name", StringType(), True),
    StructField("pr_approvalstatuscode", StringType(), True),
    StructField("pr_approvalstatus", StringType(), True),
    StructField("pr_approveddate", StringType(), True),
    StructField("pr_processingstatuscode", StringType(), True),
    StructField("pr_processingstatus", StringType(), True),
    StructField("pr_companycode", StringType(), True),
    StructField("pr_companycodedesc", StringType(), True),
    StructField("pr_plant", StringType(), True),
    StructField("pr_plantdesc", StringType(), True),
    StructField("pr_orderqty", StringType(), True),
    StructField("pr_unitofmeasure", StringType(), True),
    StructField("pr_closedindicator", StringType(), True),
    StructField("pr_deletionindicator", StringType(), True),
    StructField("recordskipindicator", StringType(), True),
    StructField("lastchangedatetime", StringType(), True)
])


# ---------------------------------------------------------------------
# 2. AUTOLOADER INGESTION CONFIG
# ---------------------------------------------------------------------

input_path = "/Volumes/abc/abc_dw_bronze/dataset/raw/"

bronze_path = "/Volumes/abc/abc_dw_bronze/dataset/bronze/procurement/po_pr/"
checkpoint_path = "/Volumes/abc/abc_dw_bronze/dataset/bronze/checkpoint/procurement/po_pr/"

df_bronze = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.inferColumnTypes", "false")
    .option("header", "true")
    .option("delimiter", ",")
    .schema(po_schema)
    .load(input_path)
)

# ---------------------------------------------------------------------
# 3. WRITE TO BRONZE DELTA
# ---------------------------------------------------------------------

(df_bronze.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .outputMode("append")
 .option("mergeSchema", "true")
 .trigger(availableNow=True)
 .toTable("abc.abc_dw_bronze.abc_dw_br_procurment_raw_data")
)
